In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import torch
import requests
import os
import argparse
from torch.utils.data import DataLoader
from collections import Counter

In [3]:
class Model(nn.Module):
    def __init__(self, dataset):
        super(Model, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3

        n_vocab = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(
        self,
        args,
    ):
        self.args = args
        self.words = self.load_words()
        self.uniq_words = self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def load_words(self):
        train_df = pd.read_csv('mobydick.csv')
        #train_df = train_df[:3000]
        text = train_df['Line'].str.cat(sep=' ')
        return text.split(' ')

    def get_uniq_words(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.args.sequence_length

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.args.sequence_length]),
            torch.tensor(self.words_indexes[index+1:index+self.args.sequence_length+1]),
        )

In [5]:
def train(dataset, model, args):
    model.train()

    dataloader = DataLoader(dataset, batch_size=args.batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(args.max_epochs):
        state_h, state_c = model.init_state(args.sequence_length)

        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            y_pred, (state_h, state_c) = model(x, (state_h, state_c))
            loss = criterion(y_pred.transpose(1, 2), y)

            state_h = state_h.detach()
            state_c = state_c.detach()

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })

In [6]:
def predict(dataset, model, text, next_words=100):
    model.eval()

    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])

    return words

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument('--max-epochs', type=int, default=100)
parser.add_argument('--batch-size', type=int, default=256)
parser.add_argument('--sequence-length', type=int, default=4)
#args = parser.parse_args()
args, unknown = parser.parse_known_args()
dataset = Dataset(args)
model = Model(dataset)

In [11]:
train_df = pd.read_csv('mobydick.csv')
train_df.head(100)

,Unnamed: 0,Line
0,0,CHAPTER 2
1,1,The Carpet-Bag
2,2,CHAPTER 3
3,3,The Spouter-Inn
4,4,CHAPTER 4
...,...,...
95,95,The Hyena
96,96,CHAPTER 50
97,97,Ahab’s Boat and Crew
98,98,Fedallah


In [12]:
train(dataset, model, args)

{'epoch': 0, 'batch': 0, 'loss': 10.335919380187988}
{'epoch': 0, 'batch': 1, 'loss': 10.280003547668457}
{'epoch': 0, 'batch': 2, 'loss': 10.236092567443848}
{'epoch': 0, 'batch': 3, 'loss': 10.175676345825195}
{'epoch': 0, 'batch': 4, 'loss': 10.080366134643555}
{'epoch': 0, 'batch': 5, 'loss': 10.032454490661621}
{'epoch': 0, 'batch': 6, 'loss': 9.611031532287598}
{'epoch': 0, 'batch': 7, 'loss': 9.2224702835083}
{'epoch': 0, 'batch': 8, 'loss': 9.837698936462402}
{'epoch': 0, 'batch': 9, 'loss': 9.28374195098877}
{'epoch': 0, 'batch': 10, 'loss': 9.174955368041992}
{'epoch': 0, 'batch': 11, 'loss': 8.725141525268555}
{'epoch': 0, 'batch': 12, 'loss': 8.471175193786621}
{'epoch': 0, 'batch': 13, 'loss': 8.437949180603027}
{'epoch': 0, 'batch': 14, 'loss': 8.3910493850708}
{'epoch': 0, 'batch': 15, 'loss': 7.9011030197143555}
{'epoch': 0, 'batch': 16, 'loss': 7.97749662399292}
{'epoch': 0, 'batch': 17, 'loss': 7.654041290283203}
{'epoch': 0, 'batch': 18, 'loss': 7.4477009773254395}
{

{'epoch': 0, 'batch': 154, 'loss': 6.660232067108154}
{'epoch': 0, 'batch': 155, 'loss': 7.168051719665527}
{'epoch': 0, 'batch': 156, 'loss': 7.212050914764404}
{'epoch': 0, 'batch': 157, 'loss': 7.411373138427734}
{'epoch': 0, 'batch': 158, 'loss': 7.166880130767822}
{'epoch': 0, 'batch': 159, 'loss': 6.744512557983398}
{'epoch': 0, 'batch': 160, 'loss': 7.136436939239502}
{'epoch': 0, 'batch': 161, 'loss': 7.030271530151367}
{'epoch': 0, 'batch': 162, 'loss': 7.155222415924072}
{'epoch': 0, 'batch': 163, 'loss': 6.858227729797363}
{'epoch': 0, 'batch': 164, 'loss': 6.665462493896484}
{'epoch': 0, 'batch': 165, 'loss': 6.981253623962402}
{'epoch': 0, 'batch': 166, 'loss': 7.136292934417725}
{'epoch': 0, 'batch': 167, 'loss': 6.851579666137695}
{'epoch': 0, 'batch': 168, 'loss': 6.867325782775879}
{'epoch': 0, 'batch': 169, 'loss': 6.765490531921387}
{'epoch': 0, 'batch': 170, 'loss': 6.880496978759766}
{'epoch': 0, 'batch': 171, 'loss': 7.343597412109375}
{'epoch': 0, 'batch': 172, '

{'epoch': 0, 'batch': 306, 'loss': 7.2242817878723145}
{'epoch': 0, 'batch': 307, 'loss': 7.0748491287231445}
{'epoch': 0, 'batch': 308, 'loss': 7.018703937530518}
{'epoch': 0, 'batch': 309, 'loss': 7.035366058349609}
{'epoch': 0, 'batch': 310, 'loss': 7.201539993286133}
{'epoch': 0, 'batch': 311, 'loss': 7.155364036560059}
{'epoch': 0, 'batch': 312, 'loss': 7.457404613494873}
{'epoch': 0, 'batch': 313, 'loss': 7.635278224945068}
{'epoch': 0, 'batch': 314, 'loss': 7.448719501495361}
{'epoch': 0, 'batch': 315, 'loss': 7.018942832946777}
{'epoch': 0, 'batch': 316, 'loss': 7.0924859046936035}
{'epoch': 0, 'batch': 317, 'loss': 7.093681812286377}
{'epoch': 0, 'batch': 318, 'loss': 7.251557350158691}
{'epoch': 0, 'batch': 319, 'loss': 7.0964579582214355}
{'epoch': 0, 'batch': 320, 'loss': 7.186403751373291}
{'epoch': 0, 'batch': 321, 'loss': 7.280016899108887}
{'epoch': 0, 'batch': 322, 'loss': 7.165194988250732}
{'epoch': 0, 'batch': 323, 'loss': 6.996649265289307}
{'epoch': 0, 'batch': 32

{'epoch': 0, 'batch': 458, 'loss': 7.1430840492248535}
{'epoch': 0, 'batch': 459, 'loss': 6.90346622467041}
{'epoch': 0, 'batch': 460, 'loss': 6.81420373916626}
{'epoch': 0, 'batch': 461, 'loss': 6.644617557525635}
{'epoch': 0, 'batch': 462, 'loss': 6.767252445220947}
{'epoch': 0, 'batch': 463, 'loss': 7.016932010650635}
{'epoch': 0, 'batch': 464, 'loss': 6.708754539489746}
{'epoch': 0, 'batch': 465, 'loss': 6.987767219543457}
{'epoch': 0, 'batch': 466, 'loss': 7.319790363311768}
{'epoch': 0, 'batch': 467, 'loss': 7.278587341308594}
{'epoch': 0, 'batch': 468, 'loss': 7.139537334442139}
{'epoch': 0, 'batch': 469, 'loss': 7.045688629150391}
{'epoch': 0, 'batch': 470, 'loss': 6.968596935272217}
{'epoch': 0, 'batch': 471, 'loss': 6.992384433746338}
{'epoch': 0, 'batch': 472, 'loss': 6.957716464996338}
{'epoch': 0, 'batch': 473, 'loss': 7.021686553955078}
{'epoch': 0, 'batch': 474, 'loss': 6.834214210510254}
{'epoch': 0, 'batch': 475, 'loss': 7.176138401031494}
{'epoch': 0, 'batch': 476, 'l

{'epoch': 0, 'batch': 610, 'loss': 7.077310562133789}
{'epoch': 0, 'batch': 611, 'loss': 7.128796577453613}
{'epoch': 0, 'batch': 612, 'loss': 7.2265424728393555}
{'epoch': 0, 'batch': 613, 'loss': 6.866949081420898}
{'epoch': 0, 'batch': 614, 'loss': 7.692000389099121}
{'epoch': 0, 'batch': 615, 'loss': 7.081242561340332}
{'epoch': 0, 'batch': 616, 'loss': 7.01769495010376}
{'epoch': 0, 'batch': 617, 'loss': 7.14919900894165}
{'epoch': 0, 'batch': 618, 'loss': 6.826390743255615}
{'epoch': 0, 'batch': 619, 'loss': 6.728329181671143}
{'epoch': 0, 'batch': 620, 'loss': 6.750149726867676}
{'epoch': 0, 'batch': 621, 'loss': 6.742939472198486}
{'epoch': 0, 'batch': 622, 'loss': 6.864295959472656}
{'epoch': 0, 'batch': 623, 'loss': 7.1252264976501465}
{'epoch': 0, 'batch': 624, 'loss': 6.772565841674805}
{'epoch': 0, 'batch': 625, 'loss': 6.655148029327393}
{'epoch': 0, 'batch': 626, 'loss': 6.83991813659668}
{'epoch': 0, 'batch': 627, 'loss': 6.848410606384277}
{'epoch': 0, 'batch': 628, 'l

...

{'epoch': 22, 'batch': 259, 'loss': 3.803457260131836}
{'epoch': 22, 'batch': 260, 'loss': 3.6432390213012695}
{'epoch': 22, 'batch': 261, 'loss': 3.360919713973999}
{'epoch': 22, 'batch': 262, 'loss': 4.007322788238525}
{'epoch': 22, 'batch': 263, 'loss': 3.5744411945343018}
{'epoch': 22, 'batch': 264, 'loss': 3.8066065311431885}
{'epoch': 22, 'batch': 265, 'loss': 3.3392763137817383}
{'epoch': 22, 'batch': 266, 'loss': 3.8474957942962646}
{'epoch': 22, 'batch': 267, 'loss': 4.239789009094238}
{'epoch': 22, 'batch': 268, 'loss': 3.896740198135376}
{'epoch': 22, 'batch': 269, 'loss': 3.9431238174438477}
{'epoch': 22, 'batch': 270, 'loss': 3.6034328937530518}
{'epoch': 22, 'batch': 271, 'loss': 4.021547317504883}
{'epoch': 22, 'batch': 272, 'loss': 3.825653314590454}
{'epoch': 22, 'batch': 273, 'loss': 4.055564880371094}
{'epoch': 22, 'batch': 274, 'loss': 4.103059768676758}
{'epoch': 22, 'batch': 275, 'loss': 3.920776844024658}
{'epoch': 22, 'batch': 276, 'loss': 4.102165222167969}
{'e

{'epoch': 22, 'batch': 408, 'loss': 4.109973907470703}
{'epoch': 22, 'batch': 409, 'loss': 4.199538230895996}
{'epoch': 22, 'batch': 410, 'loss': 4.19941520690918}
{'epoch': 22, 'batch': 411, 'loss': 3.8781800270080566}
{'epoch': 22, 'batch': 412, 'loss': 4.077629566192627}
{'epoch': 22, 'batch': 413, 'loss': 4.336363792419434}
{'epoch': 22, 'batch': 414, 'loss': 4.218297481536865}
{'epoch': 22, 'batch': 415, 'loss': 4.180250644683838}
{'epoch': 22, 'batch': 416, 'loss': 4.2902445793151855}
{'epoch': 22, 'batch': 417, 'loss': 4.008835792541504}
{'epoch': 22, 'batch': 418, 'loss': 4.011944770812988}
{'epoch': 22, 'batch': 419, 'loss': 4.164388656616211}
{'epoch': 22, 'batch': 420, 'loss': 4.326374053955078}
{'epoch': 22, 'batch': 421, 'loss': 4.207103729248047}
{'epoch': 22, 'batch': 422, 'loss': 4.168234825134277}
{'epoch': 22, 'batch': 423, 'loss': 4.173463821411133}
{'epoch': 22, 'batch': 424, 'loss': 4.0861616134643555}
{'epoch': 22, 'batch': 425, 'loss': 3.9496536254882812}
{'epoch

{'epoch': 22, 'batch': 557, 'loss': 4.057562351226807}
{'epoch': 22, 'batch': 558, 'loss': 4.271168231964111}
{'epoch': 22, 'batch': 559, 'loss': 4.062816619873047}
{'epoch': 22, 'batch': 560, 'loss': 4.154653549194336}
{'epoch': 22, 'batch': 561, 'loss': 4.0986762046813965}
{'epoch': 22, 'batch': 562, 'loss': 4.196859359741211}
{'epoch': 22, 'batch': 563, 'loss': 3.954068183898926}
{'epoch': 22, 'batch': 564, 'loss': 3.829655408859253}
{'epoch': 22, 'batch': 565, 'loss': 4.212072372436523}
{'epoch': 22, 'batch': 566, 'loss': 4.229286193847656}
{'epoch': 22, 'batch': 567, 'loss': 4.168947219848633}
{'epoch': 22, 'batch': 568, 'loss': 4.160732269287109}
{'epoch': 22, 'batch': 569, 'loss': 3.8602166175842285}
{'epoch': 22, 'batch': 570, 'loss': 3.960644483566284}
{'epoch': 22, 'batch': 571, 'loss': 4.00527811050415}
{'epoch': 22, 'batch': 572, 'loss': 3.8479666709899902}
{'epoch': 22, 'batch': 573, 'loss': 4.248292922973633}
{'epoch': 22, 'batch': 574, 'loss': 3.9564647674560547}
{'epoch

{'epoch': 22, 'batch': 706, 'loss': 4.168858051300049}
{'epoch': 22, 'batch': 707, 'loss': 3.977583646774292}
{'epoch': 22, 'batch': 708, 'loss': 3.721846580505371}
{'epoch': 22, 'batch': 709, 'loss': 3.9379889965057373}
{'epoch': 22, 'batch': 710, 'loss': 3.9589271545410156}
{'epoch': 22, 'batch': 711, 'loss': 3.9824411869049072}
{'epoch': 22, 'batch': 712, 'loss': 3.9069230556488037}
{'epoch': 22, 'batch': 713, 'loss': 3.925969362258911}
{'epoch': 22, 'batch': 714, 'loss': 3.9976351261138916}
{'epoch': 22, 'batch': 715, 'loss': 4.143765926361084}
{'epoch': 22, 'batch': 716, 'loss': 4.156560897827148}
{'epoch': 22, 'batch': 717, 'loss': 4.105273246765137}
{'epoch': 22, 'batch': 718, 'loss': 3.9825351238250732}
{'epoch': 22, 'batch': 719, 'loss': 3.84982967376709}
{'epoch': 22, 'batch': 720, 'loss': 3.9481747150421143}
{'epoch': 22, 'batch': 721, 'loss': 3.8727574348449707}
{'epoch': 22, 'batch': 722, 'loss': 3.638838768005371}
{'epoch': 22, 'batch': 723, 'loss': 3.820096254348755}
{'e

KeyboardInterrupt: 

In [13]:
torch.save(model.state_dict(), "mobydickmodel.pt")

In [14]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'conscience', 'battle', '', 'They', '', 'now', 'straightway', 'men’s', 'oil', 'as', 'I', 'know', 'bones—beware,', 'traceable', 'into', 'her', 'like', 'another', '', 'country', 'heading', 'therefore,', 'they', 'may', 'let', 'him', 'a', 'distance', 'on', 'this', 'jets', '', 'Of', 'the', 'oil', 'of', 'the', 'Guernsey-man,', 'was', 'in', 'that', 'celestial', 'words', 'was', '', 'woe', 'upon', 'his', 'number,', 'I', 'distantly', 'it', 'in', 'a', 'long,', 'stranger', 'line', 'frankly', '', 'contains', 'distant', 'bundle', '', '', '', '', '', '', 'CHAPTER', '98', '', 'Surmises', '', '', '', '', '“De', 'aye,', 'ambergris', 'said', 'Stubb', 'in', 'this', 'mysterious,', 'oath;', 'that', 'this', 'doubloon', 'is', '', 'the', 'Pope', 'who', 'turned', 'or', 'apt', 'away', 'to', 'sustain', 'any']


In [18]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'firmly', 'leaves', '', 'that', 'that', 'boat', 'should', 'be', 'received', 'somewhat', 'courses', '', 'Likewise', 'that', 'that', '', 'hard,', 'devout', 'and', 'fragrant', 'and', 'significant', 'in', 'mincing', 'it', 'coercing', 'cleansed', '', 'thousands', 'of', 'chopping', 'indeed,', 'they', 'dodges', 'so', 'being', 'then', 'is', 'the', 'lavish', '', 'One', 'Whale', 'may', 'be', 'found', 'up', 'so,', 'and', 'fill', 'the', 'same', 'barbs', 'was', 'cut', '', 'to', 'that', 'celestial', 'voyage', 'and', 'nothing', 'accelerated,', 'in', 'the', 'boat,', 'and', 'secured', '', 'from', 'his', 'symmetrical', 'race', 'in', 'two', 'whale', 'must', 'be', 'really', 'cutting', 'it', 'would', 'call', '', 'themselves', 'out', 'in', 'the', 'cause', 'of', 'spears', 'and', 'lamps', 'their', 'objects', 'illumined', 'their', '', 'plain,', 'when,']


In [35]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'individual', 'G', '', '', '', '', '', 'Thomas', 'Parke', 'd’Invilliers', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'I', '', '', '', 'I', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Mr', '', 'In', 'some', 'more', 'more', 'fact', 'the', 'dollars’', 'physical', 'addressed', 'over', 'to', 'her', 'husband', '”', '', '', '', '“Can’t', 'him,”', 'you', 'doing,', 'answered', 'in', 'Mr', '', 'McKee', 'and', 'the', 'butler', 'rather', 'heard']


In [36]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'yelling,', 'small-breasted', '“character”', 'at,', 'year,”', '“Aunt', '“Old', 'darkening', 'heady', 'heady', 'were', 'wiping', 'to', 'four', 'evening', 'face', '', '', '', '', '“You', 'did', 'how', 'so', 'exactly', 'any', 'long', '', 'Tomorrow', 'to', 'see', 'it', 'is', 'like', 'more', '', 'restlessly', 'were', 'sitting', 'and', 'her', 'rather', 'interest', 'not', 'said,', 'one', 'attention', '', 'with', 'a', 'taxi', 'alone', 'in', 'the', 'best,', 'family', 'Mrs', '', 'Wilson', 'turned', '', 'on', 'the', 'Coloured', 'nod,', 'and', 'they', 'sat', 'over', 'at', 'me', 'with', 'lovely', 'longer', 'as', 'I', '', 'get', '', 'share', 'over', 'that', 'they', 'should', 'nothing', 'to', 'make', 'more', 'breath', 'as', 'finally', 'some', '', 'slight', 'people', 'above', 'it', 'outside', 'in', 'college']


In [37]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'science—”', 'Next', 'Ohio,', 'college—one', 'Forms', 'You’d', 'questioningly', 'approved', 'flashing', 'flung', 'shoe', 'bosom', 'gravely,', 'refused,', 'slow', 'somnambulatory', 'Ripley', 'ivy,', 'autumn-leaf', 'Thus', 'confessed', 'alone,”', 'soaked', 'side—“she', 'Dear', 'Moreover', 'Reading', 'neared', 'swell', 'be', '', '', '', 'Thomas', 'Parke', 'd’Invilliers', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'inquired', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [38]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'slit', 'Get', 'Generally', 'of—oh—fling', 'degree', 'explicable', 'use', 'of', 'our', 'office', 'behind', '', 'with', 'no', 'slightly', 'glow', 'into', 'the', 'city', 'word', 'he', 'spent', 'his', 'murky', '', 'windows,', 'and', 'looked', 'back', 'from', 'overhead', '', 'But', 'it', 'held', 'off', 'with', 'a', 'life', '', 'beside', 'his', 'eyes', 'was', 'out,', 'with', 'a', 'boots', 'by', 'Mrs', '', 'Wilson', 'lay', '', 'back', 'of', 'find', 'there', 'outside', '', 'I', 'was', 'an', 'ghastly', 'men', 'with', 'an', '', 'exclamation', 'promising', 'a', 'lustreless', 'affair', '', 'She', 'had', 'apparently', 'Wilson,', 'one', 'of', 'the', 'quality', 'of', '', 'Wilson', 'seemed', 'the', 'damp', 'island', 'of', 'like', 'three', 'promise', 'as', 'it', '', 'still', 'the', 'line', 'in']


In [39]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'talked,', 'cause', 'marry', 'rugs', 'confessed', 'unexpected', 'frogs', 'yesterday', 'career', 'cars,', 'leather', 'doing', 'sea', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'General', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '6:00', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [40]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'forth', 'Those', 'frowning,', 'confidences', 'link', 'Those', 'reputation', 'chambermaids', 'restlessness', '“Come', 'broken', 'nothing', 'her', 'mind', '', 'it’s', 'frowning', '', 'and', 'it', 'has', 'me', 'when', 'he', 'shall', 'that', 'wanted', 'to', 'be', 'alone', '', '', '', '', '“I’d', 'was', 'to', 'look', 'for', 'this', 'afternoon', '”', '', '', '', '“Young', 'Baker', '', 'cried', 'explained', 'the', 'matter', '', 'I', 'went', 'when', 'I', 'get', 'that', 'I', 'was', 'going', 'to', 'let', '', 'him,', 'until', 'what', 'poor', 'Lucille', 'I', 'have', '', 'repeat,', 'been', 'likely', 'in', 'love', '', 'anybody,', 'than', 'it', 'had', 'been', 'more', '', 'Well,', 'this', 'one', 'more', 'would', 'drift', '', 'he’d', 'usually', 'much', 'alone', '', 'I', 'wanted']


In [41]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'stale,', 'forgive', 'elocution,', 'hey', 'deserting', 'steal', 'happiness', '', '', '', '', 'In', 'my', 'friend', '”', '', '', '', 'What', 'I', 'called', 'in', 'curious', 'driving', 'my', 'moment,', 'Mr', '', 'Wolfshiem', '', 'Wolfshiem', 'made', 'him', '', 'He', 'was', 'getting', 'over', 'that', 'more', '', 'island', 'between', 'referred', 'to', 'me', '', 'Her', 'name', 'was', 'printed', 'no', 'company', 'lay', 'rather', 'no', 'shoulders', 'of', '', 'confused', '', 'It', 'was', 'this', 'wheel', '', 'The', 'lady', 'lawn,', 'suddenly', '', 'dominance', 'his', 'East', 'eyes', '', '', '', '', '“My', 'dear,”', 'she', 'remarked,', 'this', 'hovering', 'about', 'three', 'say,', 'say,', 'forced', '', 'nothing', 'quickly', 'an', 'look', 'or', 'suddenly', 'it', 'like', 'a']


In [42]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'tightening', '—', 'Where', 'See', 'Say:', 'Conduct', 'thanking', 'subtle', 'other', 'garage,', 'chair', 'plan', 'their', 'silence', 'like', 'beneath', 'that', 'while', 'that—I', 'touched', 'a', 'hasty', 'places', 'like', '', 'this', 'too', 'has', 'have', 'any', 'little', 'before', '', 'He', 'had', 'shown', 'to', '', 'stop', 'there', 'with', 'the', 'funeral', 'when', 'he', 'firmly', 'in', '1915,', 'helped', 'around', '', 'the', 'particular', 'rose', 'than', 'the', 'uncertainty', 'of', 'the', 'desk—he’d', 'when', 'my', 'rose', '', 'then', 'quickly', 'waiting', 'for', 'an', 'certain', 'never', 'look', 'called', 'now,', 'and', 'they', '', 'still', 'handsome', 'he', 'might', 'made', 'a', 'newspapers,', 'his', 'car', '', '', '', '', 'Gatsby', 'here,', 'suddenly', 'up', 'Fifth', 'morning', 'that', 'they', 'had', 'never']


In [43]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'F', 'Various—”', 'composed', '', 'West', 'I', 'am', 'tied', 'on', 'the', 'room', 'after', 'her', 'two', 'left', 'yellow', 'open', '', 'into', 'the', 'glasses', 'but', 'stopping', 'over', 'a', 'little', 'idea', '', 'This', 'started', 'that', 'I', '', 'have', 'been', 'back', 'into', 'the', 'person', '', 'After', 'the', 'standard—it', 'was', 'full', 'and', '', 'Buccleuch,', 'but', 'the', 'Fourth', 'was', 'ajar', 'and', 'there', 'in', 'the', 'flutter', 'of', 'a', '', 'egg,', 'light', '', 'At', 'a', 'younger', 'and', 'the', 'earth', '', 'painted', 'veiled', 'throwing', 'her', 'over', 'on', 'an', 'beads', 'with', 'leaning', '', 'dress', 'and', 'other', 'nose', 'draws', 'the', 'sun', 'with', 'a', 'party', 'that', '', '', '', '', 'Across', 'it', '”']


In [44]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'taxicabs', 'Its', 'Thirty—the', 'Daisy’ll', 'asleep—she', 'extravagantly', 'feigned', 'ahead,”', 'fun—”', 'last', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '6:15-6:30', '”', '', '', '', '']


In [45]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'relieved', 'is—”', 'revolver', 'west,', 'legal', 'stoop', 'Bird', 'Unlike', 'questioningly', '“\u200a‘Let', 'vacuous', 'juxtaposition', 'remarked', 'dark', 'wan', 'car', '', '', '', '', 'shouted', 'at', 'such', 'that', 'hasty', 'addenda', 'old', 'sport,', 'you’ve', 'been', 'rather', 'was', 'coming', '', 'with', 'just', 'all', 'Daisy', '', 'And', 'that', 'are', 'my', 'other', 'girl,', '', 'they', 'had', 'gone', 'that', 'four', 'night', 'after', 'a', 'thrilling', 'thing', 'and', 'did', '', 'Louisville', 'who', 'was', 'privy', 'with', 'a', 'ghastly', 'five', 'empty', 'sky', 'with', 'two', '', 'unknown', 'physical', 'rose', 'over', 'along', 'the', 'stone', '', 'We', 'curtains', 'like', '', 'hard', 'on', 'a', 'edge', 'of', 'fact', 'that', 'I', 'spent', 'all', 'affection', '', 'after', 'noon', 'I', 'sounded']


In [46]:
print(predict(dataset, model, text='You are'))

['You', 'are', 'towns,', 'polish', 'prig', 'jovially', 'unforgivably', 'Moreover', 'Fifty-Ninth', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '4:30-5:00', '”', '', '', '', '', '', '', '', '', 'Baseball', 'and', 'private', 'Middle', 'a', 'matter', 'by', 'my', 'father', 'wasting', 'off', 'New']


In [88]:
print(predict(dataset, model, text='You make me'))

['You', 'make', 'me', 'voice', '', 'In', 'the', 'lawn,', 'morning', 'with', 'my', 'very', 'end', 'as', 'I', 'dirty', '', 'a', 'blue', 'body', 'half-bewildered', 'bright', 'preceded', 'its', 'well', 'in', 'the', 'garden,', '', 'of', 'my', 'windows', 'that', 'a', 'high', 'man', 'blew', 'her', 'other', 'intention', 'of', 'youth', 'who', '', 'D', 'in', 'some', 'doors', 'for', 'the', 'country', '', 'The', 'living-room', 'was', 'white', '', 'and', 'splendour', 'of', 'my', 'reaction—Gatsby,', 'like', 'the', 'new', 'steps', 'and', 'then', 'of', 'the', 'car', '', 'before', 'if', 'it', 'had', 'been', 'crowded', 'alone—fifty', 'sun', 'I', 'have', 'heard', 'the', 'end', 'and', '', 'natural', 'interest', 'into', 'it', 'and', 'morning', 'he', 'lay', 'off', 'that', 'head', 'in', 'half', 'an', '', 'men']


In [52]:
print(predict(dataset, model, text='I love'))

['I', 'love', '', 'I', 'am', 'sort', 'of', 'view', 'you’d', 'a', 'deep', 'explanations', 'and', 'her', '', 'trying', 'to', 'know', 'him,', 'perhaps', 'possessed', 'them', 'there', 'so', 'it’s', '', 'alone', '', 'It', 'had', 'never', 'seen', 'him', 'nibble', 'to', 'him', 'was', 'less', 'but', '', 'when', 'I’m', 'stronger', 'in', 'his', 'back', 'or', 'so', 'Nick,', 'with', 'there', 'as', 'Doctor', 'Haag,', '', 'more', '', 'But', 'they', 'know', 'ceased', 'friends', 'he', 'awoke', 'just', 'surprised', 'that', 'cool,', '', 'sauntered', 'more', 'of', 'dinner', 'he', 'couldn’t', 'say', '', 'If', 'the', 'figure', 'was', 'a', 'man', '', 'he', 'had', 'rather', 'feel', 'himself', 'of', 'us', 'in', 'an', 'side', 'in', 'one', 'in', 'a', 'determination', '', 'in', 'a']


In [90]:
print(predict(dataset, model, text='I am'))

['I', 'am', '', '', '', '', '7:15-8:15', 'in', 'a', 'note,', 'an', 'evening', 'began', 'now', 'there', 'was', 'a', 'promise', 'in', 'a', '', 'pool', 'and', 'I', 'should', 'get', 'out', 'of', 'my', 'series', '', 'Probably', 'I', 'live', '', 'any', 'own', 'girl', 'she', 'had', 'run', 'an', 'checker', 'to', 'waves,', '', 'my', 'share', 'from', 'privileged', 'specialists,', 'a', 'man', '”', '', '', '', '“Listen,', 'they', 'is', 'gone', 'for', 'a', 'party', '”', 'She', 'took', 'up', 'my', 'taxi', 'and', 'looked', '', 'there', 'was', 'an', 'hour', 'ago', 'and', 'did', 'it', 'might', '“got', 'a', 'communicative', '', 'that', 'happened', 'around', 'my', 'country', 'until', 'Mrs', '', 'Wilson', 'suit', 'opening', 'the', 'certain', '', 'hardy', 'turned']


In [105]:
print(predict(dataset, model, text='I am'))

['I', 'am', 'hand', '', '', '', '', 'His', 'eyes', 'unmistakably', 'dog-leash,', 'broke', 'from', 'wide', 'and,', 'and', 'what', 'she', 'knew', '', 'you', 'firmly', 'he', 'had', 'far', 'again', 'in', 'a', 'drizzle', '', 'I', 'heard', 'it', 'necessary', 'to', 'kill', 'me,', 'for', '', 'rooms', 'his', 'words', 'and', 'it', 'stopped', 'from', 'his', 'house', 'before', 'I', 'left', 'another', '', '', '', '', '“The', 'voice', 'son-of-a-bitch,”', 'my', 'book', 'she', 'said', '', '“You', 'give', 'me', 'everything', '”', '', '', '', 'A', 'shock', 'that', 'was', 'almost', 'white—changed', 'over', 'the', 'table', 'as', 'I', 'knew', '', 'at', 'its', 'most', 'respectful', 'in', 'an', 'abortive', '“Hm', 'in', 'blue', '', 'In', 'the', 'railroad', 'warm', 'old', 'girls']


In [113]:
print(predict(dataset, model, text='I love'))

['I', 'love', '', 'you’re', 'one', 'of', 'the', 'desk,”', 'that', 'long', 'lady', 'moved', 'standing', 'in', 'a', '', 'front', 'of', 'which', 'are', 'me', 'every', 'young', 'before', 'she', 'shouted', 'slightly', '', 'together', '”', '', '', '', 'Most', 'of', 'West', 'Egg,', 'the—well,', 'this', 'bursts', 'and', 'grey', 'cars', 'identical', 'his', '', 'grave', 'or', 'along', 'that', 'Gatsby', 'of', 'already', 'share', 'were', '', 'tipped', 'under', 'an', 'whip', 'and', 'leaning', 'with', 'cream-coloured', 'chiffon,', 'close', 'in', '', 'the', 'mass', 'of', 'the', 'son’s', '', 'She', 'must', 'came', 'in', 'his', 'face', 'in', 'his', 'grandfather’s', '', 'wet', 'immediately', 'upon', 'the', 'black', 'leaf,', 'from', 'its', 'shoulder', 'sank', 'by', 'breathless,', '', 'colour', 'on', 'the', 'cheerful', 'window,']


In [114]:
print(predict(dataset, model, text='I love'))

['I', 'love', '', 'uttered', 'it', '”', '', '', '', 'Evidently', 'it’s', 'a', 'obviously', 'ends', 'of', 'his', 'police', 'but', 'if', 'it', 'did', 'you', 'some', 'most', '', 'George', 'Mrs', '', 'Wilson', 'mentioned', 'up,', 'and', 'they', 'was', 'rooms', 'in', '', 'whomsoever', 'we', 'all', 'the', 'time', '', 'Some', 'kind', 'of', 'printed', 'the', 'Catholic,', '', 'who', 'was', 'going', 'to', 'curious', 'before', 'at', 'my', 'front', 'way', 'and', 'it', 'was', 'too', '', 'with', 'the', 'policeman,', 'but', 'I', 'expected,', 'his', 'head', 'by', 'a', 'simplicity', 'of', 'indifferent', '', 'sauntered', 'at', 'the', 'veranda', '', 'The', 'little', 'ashen', 'moved', 'over', 'up', 'into', '', 'him', 'so', 'since', 'I', 'drove', 'over', 'with', 'my', 'pale,', '‘All']


In [147]:
print(predict(dataset, model, text='Love you'))

['Love', 'you', 'arm', '', '', '', '', 'It', 'was', 'across', 'the', 'dog-leash', '', '', '', '', 'We', 'waited', 'for', 'up', 'in', 'the', 'funeral', 'along', 'the', 'gutter', 'he', 'was', '', 'breaking', 'from', 'the', 'son', 'of', 'me,', '', 'He', 'ate', 'more', 'incessantly', 'in', 'a', 'blurred', 'glasses,', 'and', '', 'he', 'had', 'stood', 'out', 'on', 'the', 'light', 'with', 'the', 'evening', 'of', 'half', '', 'whisky', '', 'His', 'night', 'toward', 'the', 'world', '', '', '', '', 'The', 'grey,', 'wife', 'supper—there', 'were', 'over', 'by', 'through', 'some', 'wedding', 'up', '', 'through', 'wiped', 'a', 'house', 'of', 'one', 'conversation', 'said', 'all', 'feet', '', '', '', '', '“My', 'told', 'each', 'pardon,”', 'Gatsby', '”']


In [159]:
print(predict(dataset, model, text='happy birthday'))

['happy', 'birthday', '', '', '', '', 'He', 'eyes', 'said', 'driving', '', 'I’m', 'a', 'word,', 'beside', 'me', 'asked', 'if', 'I', 'used', 'to', 'think', 'when', 'we', 'were', 'related', '', 'to', 'run', 'different', 'his', 'house,', 'imperatively', 'before', 'everybody', 'real', 'even', '', 'forever;', 'you', 'pick', 'up', 'about', 'Mrs', '', 'Wilson', '—”', '', '', '', '', '“You', 'could', 'know', 'to,', 'how', 'he', 'killed', 'the', 'butler', 'with', 'me,', 'of', '', 'restlessly', 'but', 'the', 'man', 'than', 'suggested,', 'in', 'Miss', 'Baker', 'in', 'his', '', 'raised', 'nose', 'as', 'the', 'efforts', 'was', 'white', 'more', 'but', 'he', 'seemed', 'a', '', 'permanent', 'eyes', 'at', 'the', 'water,', 'on', 'the', 'air', '', 'I', 'waited,', 'and', 'then']


In [119]:
'Love you square crowded'

'Love you square crowded'

In [ ]:
Love you scared 7:15-7:18
Love you dizzy, bright, couch wiping